# moving to gpu

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [12]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        t = self.out(t)

        return t


In [4]:
t = torch.ones([1, 1, 28, 28])
network = Network()


In [5]:
t = t.cuda()
network = network.cuda()

In [6]:
gpu_pred = network(t)
gpu_pred.device

device(type='cuda', index=0)

# moving to cpu

In [7]:
t = t.cpu()
network = network.cpu()

In [8]:
cpu_pred = network(t)
cpu_pred.device

device(type='cpu')

In [9]:
network = Network()


In [10]:
for name, param in network.named_parameters():
    print(name, '\t\t', param.shape)


conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


In [15]:
for n, p in network.named_parameters():
    print(p.device, '', n)


cpu  conv1.weight
cpu  conv1.bias
cpu  conv2.weight
cpu  conv2.bias
cpu  fc1.weight
cpu  fc1.bias
cpu  fc2.weight
cpu  fc2.bias
cpu  out.weight
cpu  out.bias


In [17]:
network.to('cuda')

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [18]:
for n, p in network.named_parameters():
    print(p.device, '', n)


cuda:0  conv1.weight
cuda:0  conv1.bias
cuda:0  conv2.weight
cuda:0  conv2.bias
cuda:0  fc1.weight
cuda:0  fc1.bias
cuda:0  fc2.weight
cuda:0  fc2.bias
cuda:0  out.weight
cuda:0  out.bias


In [19]:
sample = torch.ones(1, 1, 28, 28)
sample.shape


torch.Size([1, 1, 28, 28])

In [20]:
try:
    network(sample)
except Exception as e:
    print(e)


Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor


In [21]:
torch.cuda.is_available()


True

In [22]:
try:
    pred = network(sample.to('cuda'))
    print(pred)
except Exception as e:
    print(e)


tensor([[-0.0200,  0.0138, -0.1065, -0.0161, -0.1208, -0.0923, -0.0070,  0.1021,
         -0.0414,  0.0174]], device='cuda:0', grad_fn=<AddmmBackward>)
